In [3]:
import laser_lib
import numpy as np
import math
import time


In [5]:
queue = laser_lib.DacQueue()

Found  1 Helios DACs


In [6]:
def calculate_sample_energy(sample):
    # Calculate the energy of the sample
    energy = np.sum(np.square(sample))
    return energy

def energy_wave(sample):
    # Calculate the energy of the sample
    energy = np.square(sample)
    return energy

def stream_bytes_to_numpy(data, num_channels):
     # Convert bytes to a NumPy array of floats
    audio_data = np.frombuffer(data, dtype=np.int16)
    
    # Normalize the audio data to the range [-1, 1]
    audio_data = audio_data.astype(np.float32) / np.iinfo(np.int16).max
    
    # Reshape the data to have one row per channel
    audio_data = audio_data.reshape(-1, num_channels)

    return audio_data

In [12]:
# Integrate the music with the laser
import wave
import pyaudio

T = 200 # at 30kpps this is 30x a second
color_arr = np.ones((T,3))
theta = np.linspace(0, 2*np.pi, T)
freq = 2
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(freq*theta)
pos_arr[:, 1] = np.sin(freq*theta)

def play_audio_file(file_path, chunk_size=2*1024):
    wf = wave.open(file_path, 'rb')
    p = pyaudio.PyAudio()

    num_channels = wf.getnchannels()
    # Open output stream
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=num_channels,
                    rate=wf.getframerate(),
                    output=True)

    data = wf.readframes(chunk_size)


    max_energy = 0
    # Play the file chunk by chunk
    while data:
        stream.write(data)
        data = wf.readframes(chunk_size)
        energy = calculate_sample_energy(stream_bytes_to_numpy(data, num_channels))
        if(energy > max_energy):
            max_energy = energy
        energy_normed = energy/max_energy
        queue.submit(energy_normed*pos_arr, color_arr, debug=False)
        

    # Close the stream and terminate PyAudio
    stream.close()
    p.terminate()

file_path = "PurpleSun.wav"
play_audio_file(file_path)


KeyboardInterrupt: 

In [41]:
# Process the timestamped file
import csv
import numpy as np

def find_nearest_timestamp_index(arr_timestamps, target_timestamp):
    min_diff = float('inf')
    nearest_index = -1

    diff = abs(arr_timestamps - target_timestamp)
    nearest_index = np.argmin(diff)
    best_timestamp = arr_timestamps[nearest_index]

    return nearest_index, best_timestamp

def process_csv_file(file_path, arr_timestamps):
    indicator = np.zeros_like(arr_timestamps)
    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter=' ')
        for row in reader:
            print(row[0].split('\t'))
            row = row[0].split('\t')
            start_time = float(row[0])
            end_time = float(row[1])
            event_type = row[2]

            start_index, best_timestamp = find_nearest_timestamp_index(arr_timestamps, start_time)
            end_index, best_timestamp = find_nearest_timestamp_index(arr_timestamps, end_time)

            print(start_index, end_index, best_timestamp)
            if start_index != -1 and end_index != -1:
                indicator[start_index:end_index] = 1
    return indicator

# Example usage
num_seconds = 86400  # Number of seconds in a day
timestamps = [0] * num_seconds
arr_timestamps = np.linspace(0, 1000, 20919)

csv_file_path = 'beat_timestamps.txt'
indicator = process_csv_file(csv_file_path, arr_timestamps)

print(indicator)

['1.489000', '1.489000', 'B']
31 31 1.4819772444784396
['3.093000', '3.093000', 'B']
65 65 3.1073716416483412
['4.689000', '4.689000', 'B']
98 98 4.684960321254422
['6.305000', '6.305000', 'B']
132 132 6.310354718424324
['6.695000', '6.695000', 'B']
140 140 6.692800458934888
['7.888000', '7.888000', 'B']
165 165 7.887943398030404
['7.978000', '7.978000', 'B']
167 167 7.9835548331580455
['9.504000', '9.504000', 'B']
199 199 9.513337795200306
['11.102000', '11.102000', 'B']
232 232 11.090926474806388
['11.185000', '11.185000', 'B']
234 234 11.186537909934028
['12.691000', '12.691000', 'B']
265 265 12.668515154412468
['12.737000', '12.737000', 'B']
266 266 12.716320871976288
['13.126000', '13.126000', 'B']
275 275 13.146572330050674
['14.132000', '14.132000', 'B']
296 296 14.150492398890908
['14.287000', '14.287000', 'B']
299 299 14.29390955158237
['15.903000', '15.903000', 'B']
333 333 15.91930394875227
['17.493000', '17.493000', 'B']
366 366 17.496892628358353
['17.502000', '17.502000',

In [53]:
# Integrate the music with the laser
import wave
import pyaudio

T = 200 # at 30kpps this is 30x a second
color_arr = np.ones((T,3))
theta = np.linspace(0, 2*np.pi, T)
freq = 2
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(freq*theta)
pos_arr[:, 1] = np.sin(freq*theta)

file_path = "PurpleSun.wav"
chunk_size=2*1024
wf = wave.open(file_path, 'rb')
p = pyaudio.PyAudio()

# Channels in the audio file.
num_channels = wf.getnchannels()
# Number of polls per second (for 2ch, this is 2x the samplerate)
framerate = wf.getframerate()
# Total number of frames in the file.
num_frames = wf.getnframes()
# The total runtime of the audio file.
runtime = num_frames / framerate
# Temporal resolution of the channels.
samplerate = framerate // num_channels
# 
num_time_samples = num_frames // num_channels

#sample_timestamps = np.zeros(int(runtime * samplerate) + 1)
sample_timestamps = np.linspace(0, runtime, num_time_samples)
indicator = process_csv_file(csv_file_path, sample_timestamps)


time_per_chunk = chunk_size / samplerate 
print('Channels: {0}\tFramerate: {1} Hz\tSamplerate: {2} Hz\tSamplesPerChunk: {3}\tTimePerChunk: {4}s\tNumFrames: {5}\tTrackTime: {6}s'.format(
    num_channels, framerate, samplerate, chunk_size, time_per_chunk, num_frames, runtime))

# Open output stream
stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=num_channels,
                rate=framerate,
                output=True)

data = wf.readframes(chunk_size)

chunk_idx = 0
max_energy = 0
# Play the file chunk by chunk
while data:
    print('Time: {0:2.2f}s'.format(chunk_idx*time_per_chunk))
    # Get the chunk of indicator 
    color_scaler = indicator[chunk_idx * chunk_size : (chunk_idx + 1)*chunk_size]
    print(color)
    stream.write(data)
    data_bytes = wf.readframes(chunk_size)
    data_np = stream_bytes_to_numpy(data, num_channels)
    print(data_np.shape)
    energy = calculate_sample_energy(data_np)
    if(energy > max_energy):
        max_energy = energy
    energy_normed = energy/max_energy
    queue.submit(energy_normed*pos_arr, color_scaler*color_arr, debug=False)
    chunk_idx += 1
    

# Close the stream and terminate PyAudio
stream.close()
p.terminate()





['1.489000', '1.489000', 'B']
32832 32832 1.4889800080375746
['3.093000', '3.093000', 'B']
68201 68201 3.093016737578296
['4.689000', '4.689000', 'B']
103392 103392 4.688980902504292
['6.305000', '6.305000', 'B']
139025 139025 6.304990424507304
['6.695000', '6.695000', 'B']
147625 147625 6.695013209263735
['7.888000', '7.888000', 'B']
173930 173930 7.887984064265818
['7.978000', '7.978000', 'B']
175915 175915 7.978006765165994
['9.504000', '9.504000', 'B']
209563 209563 9.50399358626883
['11.102000', '11.102000', 'B']
244799 244799 11.101998568091807
['11.185000', '11.185000', 'B']
246629 246629 11.184991788569047
['12.691000', '12.691000', 'B']
279836 279836 12.690978604081467
['12.737000', '12.737000', 'B']
280851 280851 12.737010362980046
['13.126000', '13.126000', 'B']
289428 289428 13.125990063544686
['14.132000', '14.132000', 'B']
311611 311611 14.132022090783279
['14.287000', '14.287000', 'B']
315028 315028 14.286988120494062
['15.903000', '15.903000', 'B']
350661 350661 15.9029

ValueError: operands could not be broadcast together with shapes (2048,) (200,3) 

In [34]:
wf.__dict__

NameError: name 'wf' is not defined

In [ ]:
scipy.interpolate.interp1d(np.arange(10), 